# PDF Extraktion mit Docling

Test mit Docling

## Installation

Librarys installieren

In [3]:
!pip install docling pillow python-docx

29.26s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


## Import Libraries

In [4]:
from docling.document_converter import DocumentConverter


from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from pathlib import Path

/Users/alice/Repositorys/llm-project-gregus-karbassi/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## PDF Pipeline Optionen konfigurieren (optional)

## PDF und Ausgabe-Datei angeben

In [10]:
# Pfad zu PDF-Datei und Ausgabe-Datei festlegen

extract = "origin"
#extract = "geschichtsbuch"

pdf_path_origin = "docs/Origin by Dan Brown.pdf" 
pdf_path_geschichtsbuch = "docs/Illustrierte Schweizer Geschichte für Sekundar- und Mittelschulen.pdf"  # Passe den Pfad an deine Datei an


if extract == "geschichtsbuch":
    pdf_path = pdf_path_geschichtsbuch 
else:
    pdf_path = pdf_path_origin

if extract == "geschichtsbuch": 
    output_path = "docs/extracted_content_geschichtsbuch.md"
    skip_pages = 12  # Erste 12 Seiten überspringen
else:
    output_path = "docs/extracted_content_dan_brown.md"
    skip_pages = 11  # Erste 11 Seiten überspringen


print(f"pdf_path is: {pdf_path}")
print(f"skip_pages: {skip_pages}")
print(f"output_path is: {output_path}")

pdf_path is: docs/Origin by Dan Brown.pdf
skip_pages: 11
output_path is: docs/extracted_content_dan_brown.md


## PDF Extraktion

In [11]:
# DocumentConverter initialisieren
converter = DocumentConverter()

# PDF konvertieren
result = converter.convert(pdf_path)

# Ergebnis ausgeben
print("Konvertierung erfolgreich!")
print(f"Anzahl der Seiten: {len(result.document.pages)}")

Konvertierung erfolgreich!
Anzahl der Seiten: 480


## Text extrahieren

In [12]:
# Text seitenweise mit Seiteninformation extrahieren
import re

# Sammle alle Text-Elemente mit ihren Seiteninformationen
markdown_parts = []
current_page = None

# Iteriere über alle Textelemente im Dokument
for element, level in result.document.iterate_items():
    # Prüfe, ob das Element eine Seitennummer hat
    if hasattr(element, 'prov') and element.prov:
        for prov in element.prov:
            if hasattr(prov, 'page_no'):
                page_no = prov.page_no
                
                # Erste N Seiten überspringen (abhängig von skip_pages)
                if page_no < skip_pages:
                    continue
                
                # Wenn wir eine neue Seite erreichen, füge Seiten-Header hinzu
                if page_no != current_page:
                    current_page = page_no
                    page_num = page_no  # 1-basierte Seitennummer
                    markdown_parts.append(f"\n\n## Seite {page_num}\n\n")
                
                # Füge den Text des Elements hinzu
                if hasattr(element, 'export_to_markdown'):
                    try:
                        # Versuche mit Dokument-Argument
                        markdown_parts.append(element.export_to_markdown(result.document))
                        markdown_parts.append("\n")
                    except:
                        # Falls das nicht funktioniert, versuche ohne Argument
                        try:
                            markdown_parts.append(element.export_to_markdown())
                            markdown_parts.append("\n")
                        except:
                            # Falls auch das nicht funktioniert, verwende text falls vorhanden
                            if hasattr(element, 'text'):
                                markdown_parts.append(element.text)
                                markdown_parts.append("\n")
                elif hasattr(element, 'text'):
                    markdown_parts.append(element.text)
                    markdown_parts.append("\n")
                
                break  # Nur erste Provenance verwenden

# Alle Teile zusammenfügen
markdown_text = "".join(markdown_parts)

# Silbentrennzeichen entfernen, aber Struktur beibehalten
# Silbentrennung am Zeilenende: "Wort¬\nFortsetzung" wird zu "WortFortsetzung"
markdown_text_postprocessed = re.sub(r'¬\s*\n\s*', '', markdown_text)
# Übrige ¬ Zeichen entfernen (falls nicht am Zeilenende)
markdown_text_postprocessed = markdown_text_postprocessed.replace('¬', '')
# Mehrfache aufeinanderfolgende Leerzeichen in derselben Zeile reduzieren
markdown_text_postprocessed = re.sub(r'  +', ' ', markdown_text_postprocessed)

pages_extracted = len([k for k in result.document.pages.keys() if k >= skip_pages])
print("Extrahierter Text (Markdown-Format):")
print("=" * 50)
print(f"Seiten extrahiert: {pages_extracted} (ab Seite {skip_pages + 1})")
print("=" * 50)
print(markdown_text_postprocessed[:1000])  # Ersten 1000 Zeichen anzeigen
print("...")
print(markdown_text_postprocessed[-500:])  # Letzten 500 Zeichen anzeigen

Extrahierter Text (Markdown-Format):
Seiten extrahiert: 470 (ab Seite 12)


## Seite 11

PROLOGUE
AS THE ANCIENT cogwheel train clawed its way up the dizzying incline, Edmond Kirsch surveyed the jagged mountaintop above him. In the distance, built into the face of a sheer cliff, the massive stone monastery seemed to hang in space, as if magically fused to the vertical precipice.
This timeless sanctuary in Catalonia, Spain, had endured the relentless pull of gravity for more than four centuries, never slipping from its original purpose: to insulate its occupants from the modern world.
Ironically, they will now be the first to learn the truth , Kirsch thought, wondering how they would react. Historically, the most dangerous men on earth were men of God … especially when their gods became threatened. And I am about to hurl a flaming spear into a hornets' nest.
When the train reached the mountaintop, Kirsch saw a solitary figure waiting for him on the platform. The wizened skeleton of a ma

## Datei speichern

In [13]:
# Als Markdown-Datei speichern
with open(output_path, "w", encoding="utf-8") as f:
    f.write(markdown_text_postprocessed)

print(f"Text wurde in '{output_path}' gespeichert")

Text wurde in 'docs/extracted_content_dan_brown.md' gespeichert
